# ローン


In [ ]:
import sys
sys.path.append('..')

from fpyjp.schemas.balance import AssetLiabilitySchema
from fpyjp.schemas.loan import Loan
from fpyjp.core.interest_factor import InterestFactor
from fpyjp.core.balance_simulator import AssetLiabilitySimulator

## 元金均等返済

In [ ]:
rate = 0.05
time_period = 5
amount = 1

In [ ]:
AssetLiabilitySimulator(
    initial_cash_balance = 0,
    initial_al_balance = -amount,
    initial_al_book_balance = -amount,
    initial_price = 1,
    rate = 0,
    cash_inflow_per_unit = rate,
    capital_cash_inflow_before_tax = 0,
    cash_outflow = [amount / time_period] * time_period,
    income_gain_tax_rate = 0,
    capital_gain_tax_rate = 0,
    allow_negative_unit=True
).simulate(n_periods=time_period)

In [ ]:
al_schema = AssetLiabilitySchema(
    price=1,
    unit=-1,
    balance=-1,
    book_balance=-1,
    cashinflow_per_unit=rate,
    rate=0,
    allow_negative_unit=True
)
AssetLiabilitySimulator(
    al_schema=al_schema,
    initial_cash_balance = 0,
    capital_cash_inflow_before_tax = 0,
    cash_outflow = [amount / time_period] * time_period,
    income_gain_tax_rate = 0,
    capital_gain_tax_rate = 0,
).simulate(n_periods=time_period)

## 元利均等返済

In [ ]:
rate = 0.05
time_period = 5
amount = 1

In [ ]:
AssetLiabilitySimulator(
    initial_cash_balance = 0,
    initial_al_balance = -1,
    initial_al_book_balance = -1,
    initial_price = 1,
    rate = rate,
    cash_inflow_per_unit = 0,
    capital_cash_inflow_before_tax = 0,
    cash_outflow = [0] + [InterestFactor(rate=rate, time_period=time_period, amount=amount).calculate_capital_recovery()] * time_period,
    income_gain_tax_rate = 0,
    capital_gain_tax_rate = 0,
    allow_negative_unit=True
).simulate(n_periods=time_period+1)

In [ ]:
al_schema = AssetLiabilitySchema(
    price=1,
    unit=-1,
    balance=-1,
    book_balance=-1,
    cashinflow_per_unit=0,
    rate=rate,
    allow_negative_unit=True
)
AssetLiabilitySimulator(
    al_schema=al_schema,
    initial_cash_balance = 0,
    capital_cash_inflow_before_tax =  0,
    cash_outflow =[0] + [InterestFactor(rate=rate, time_period=time_period, amount=amount).calculate_capital_recovery()] * time_period,
    income_gain_tax_rate = 0,
    capital_gain_tax_rate = 0,
).simulate(n_periods=time_period+1)

## 新規住宅ローンのシミュレーション

https://www.simulation.jhf.go.jp/flat35/kinri/index.php/rates/top/


### 元金均等返済シミュレーション

In [ ]:
# 新規フラット35住宅ローンの設定
new_mortgage_loan = Loan(
    name="フラット35",
    interest_rate=0.0198,  # 年利1.98%
    principal=80_000_000,  # 8000万円
    remaining_balance=80_000_000,  # 新規なので元本と同じ
    total_term_months=420,  # 35年
    remaining_term_months=420,  # 新規なので満期と同じ
    repayment_method="equal_principal",  # 元金均等返済
    interest_rate_type="fixed",
    payment_frequency=1,  # 月次返済
    repayment_amount=None  # 計算で求める
)
new_mortgage_loan

In [ ]:

monthly_rate = new_mortgage_loan.interest_rate / 12
monthly_repayment_amount = new_mortgage_loan.remaining_balance / new_mortgage_loan.remaining_term_months

In [ ]:
al_schema = AssetLiabilitySchema(
    name=new_mortgage_loan.name,
    price=1,
    unit=-new_mortgage_loan.remaining_balance,
    balance=-new_mortgage_loan.remaining_balance,
    book_balance=-new_mortgage_loan.remaining_balance,
    cashinflow_per_unit=monthly_rate,
    rate=0,
    allow_negative_unit=True
)
al_schema

In [ ]:
AssetLiabilitySimulator(
    al_schema=al_schema,
    initial_cash_balance = 0,
    capital_cash_inflow_before_tax = 0,
    cash_outflow = [monthly_repayment_amount] * new_mortgage_loan.remaining_term_months,
    income_gain_tax_rate = 0,
    capital_gain_tax_rate = 0,
).simulate(n_periods=new_mortgage_loan.remaining_term_months)

### 元利均等返済シミュレーション

In [ ]:
# 新規フラット35住宅ローンの設定
new_mortgage_loan = Loan(
    name="フラット35",
    interest_rate=0.0198,  # 年利1.98%
    principal=80_000_000,  # 8000万円
    remaining_balance=80_000_000,  # 新規なので元本と同じ
    total_term_months=420,  # 35年
    remaining_term_months=420,  # 新規なので満期と同じ
    repayment_method="equal_payment",  # 元金均等返済
    interest_rate_type="fixed",
    payment_frequency=1,  # 月次返済
    repayment_amount=None  # 計算で求める
)
new_mortgage_loan

In [ ]:
monthly_rate = new_mortgage_loan.interest_rate / 12

In [ ]:
al_schema = AssetLiabilitySchema(
    name=new_mortgage_loan.name,
    price=1,
    unit=-new_mortgage_loan.remaining_balance,
    balance=-new_mortgage_loan.remaining_balance,
    book_balance=-new_mortgage_loan.remaining_balance,
    cashinflow_per_unit=0,
    rate=monthly_rate,
    allow_negative_unit=True
)
al_schema

In [ ]:
InterestFactor(rate=monthly_rate, time_period=new_mortgage_loan.remaining_term_months, amount=new_mortgage_loan.remaining_balance).calculate_capital_recovery()

In [ ]:
AssetLiabilitySimulator(
    al_schema=al_schema,
    initial_cash_balance = 0,
    capital_cash_inflow_before_tax =  0,
    cash_outflow =[0] + [InterestFactor(rate=monthly_rate, time_period=new_mortgage_loan.remaining_term_months, amount=new_mortgage_loan.remaining_balance).calculate_capital_recovery()] * new_mortgage_loan.remaining_term_months,
    income_gain_tax_rate = 0,
    capital_gain_tax_rate = 0,
).simulate(n_periods=new_mortgage_loan.remaining_term_months+1)


## 途中からの住宅ローンのシミュレーション

### 元金均等返済シミュレーション

In [ ]:
# 新規フラット35住宅ローンの設定
existing_mortgage_loan = Loan(
    name="フラット35",
    interest_rate=0.0198,  # 年利1.98%
    principal=80_000_000,  # 8000万円
    remaining_balance=30_000_000,  # 
    total_term_months=420,  # 35年
    remaining_term_months=120,  # 
    repayment_method="equal_principal",  # 元金均等返済
    interest_rate_type="fixed",
    payment_frequency=1,  # 月次返済
    repayment_amount=None  # 計算で求める
)
existing_mortgage_loan

In [ ]:

monthly_rate = existing_mortgage_loan.interest_rate / 12
monthly_repayment_amount = existing_mortgage_loan.remaining_balance / existing_mortgage_loan.remaining_term_months

In [ ]:
al_schema = AssetLiabilitySchema(
    name=existing_mortgage_loan.name,
    price=1,
    unit=-existing_mortgage_loan.remaining_balance,
    balance=-existing_mortgage_loan.remaining_balance,
    book_balance=-existing_mortgage_loan.remaining_balance,
    cashinflow_per_unit=monthly_rate,
    rate=0,
    allow_negative_unit=True
)
al_schema

In [ ]:
AssetLiabilitySimulator(
    al_schema=al_schema,
    initial_cash_balance = 0,
    capital_cash_inflow_before_tax = 0,
    cash_outflow = [monthly_repayment_amount] * existing_mortgage_loan.remaining_term_months,
    income_gain_tax_rate = 0,
    capital_gain_tax_rate = 0,
).simulate(n_periods=existing_mortgage_loan.remaining_term_months)

### 元利均等返済シミュレーション

In [ ]:
# 新規フラット35住宅ローンの設定
existing_mortgage_loan = Loan(
    name="フラット35",
    interest_rate=0.0198,  # 年利1.98%
    principal=80_000_000,  # 8000万円
    remaining_balance=30_000_000,  # 
    total_term_months=420,  # 35年
    remaining_term_months=120,  # 
    repayment_method="equal_payment",  # 元利均等返済
    interest_rate_type="fixed",
    payment_frequency=1,  # 月次返済
    repayment_amount=None  # 計算で求める
)
existing_mortgage_loan

In [ ]:
monthly_rate = existing_mortgage_loan.interest_rate / 12

In [ ]:
al_schema = AssetLiabilitySchema(
    name=existing_mortgage_loan.name,
    price=1,
    unit=-existing_mortgage_loan.remaining_balance,
    balance=-existing_mortgage_loan.remaining_balance,
    book_balance=-existing_mortgage_loan.remaining_balance,
    cashinflow_per_unit=0,
    rate=monthly_rate,
    allow_negative_unit=True
)
al_schema

In [ ]:
AssetLiabilitySimulator(
    al_schema=al_schema,
    initial_cash_balance = 0,
    capital_cash_inflow_before_tax =  0,
    cash_outflow =[0] + [InterestFactor(rate=monthly_rate, time_period=existing_mortgage_loan.remaining_term_months, amount=existing_mortgage_loan.remaining_balance).calculate_capital_recovery()] * existing_mortgage_loan.remaining_term_months,
    income_gain_tax_rate = 0,
    capital_gain_tax_rate = 0,
).simulate(n_periods=existing_mortgage_loan.remaining_term_months+1)